## Import Library

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from sentence_transformers import SentenceTransformer
import torch
from annoy import AnnoyIndex
%matplotlib inline
%reload_ext autoreload
%autoreload 2

## Import Data

In [2]:
dataPath = Path("../data/shopify/")
apps_df = pd.read_csv(dataPath/"shopify_apps.csv")
apps_df.head(10)

,app_id,url,title,description,tagline,category
0,9e4748a9-7eda-4814-83b6-0537d44152b1,https://apps.shopify.com/translate,Panda Language Translate,Panda Language Translation improve your busine...,Translate your store into multiple languages,Store design
1,d1476138-a608-4bb9-8d39-b30f3ca7617d,https://apps.shopify.com/instant-brand-page,Instant Brand Page,Having a brand page gives your shoppers a quic...,A-Z Brand Index Page and Favourites Slider,Store design
2,d6e49a3c-2f9f-4bfa-8c26-5d024faf2241,https://apps.shopify.com/powr-faq,FAQ Accordion | Help Center,"Create, organize and display Frequently Asked ...","FAQ page, FAQ accordion menu for product Info ...","Store design,Customer support"
3,0ef0087f-3ae5-4dbc-84e0-193b576d82ed,https://apps.shopify.com/mps-promote-me,Promote Me | Many apps in one,Promote Me app is a bundle of apps that includ...,"Spin Wheel,Currency Converter,Quick ATC Button...","Store design,Sales and conversion optimization"
4,7aac2a1f-ff03-4f38-aeb7-7619403a6f05,https://apps.shopify.com/installify-boost-mobi...,Instalify,We are trusted by hundreds of Shopify and Shop...,Supercharge Your Mobile App Installs,Store design
5,c13bfb7f-8b5a-40c6-a338-dbdec5cfd130,https://apps.shopify.com/easy-product-feed,EASY product feed,There are a lot of product feed apps out there...,Get your products listed on Google Shopping th...,Marketing
6,6a71634f-f94f-498d-8713-d2b01ca90917,https://apps.shopify.com/posbill-connect,PosBill Connect,PosBill Connect - the perfection connection to...,Connect your Webshop to the solutions of PosBill.,"Orders and shipping,Inventory management"
7,da50f0bf-d116-46a0-b0c1-0d90c05d8ffe,https://apps.shopify.com/bulk-fulfill,Bulk Fulfill,Upload to fulfill: \n Bulk Fulfill by Upatra a...,Reduce wasted time - Automatically sync your t...,"Orders and shipping,Productivity"
8,89735c3c-4d25-40f2-8b54-150f59cfb099,https://apps.shopify.com/show-price-in-btc,Show Price in BTC,Simply convert your store's prices into ANY cu...,A better currency converter app. Premium featu...,Store design
9,3c89c108-4858-4b07-893a-460ea5a0d91a,https://apps.shopify.com/product-copy,POKY ‑ Product Importer,Poky \n Allows you to import/copy products fro...,Copy / Import products from any Shop store wit...,"Productivity,Finding and adding products"


## Step1: Encoding the App description

We are going to use app descriptions to find similar Shopify apps. In order to do that we need to create emeddings from the description using transformer model. We will be using sentence-transformers/paraphrase-xlm-r-multilingual-v1 model using sentence-transformers library.

In [5]:
## Initiate Model
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
model.to(torch.device("cuda"))

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [6]:
## Getting Embedding for app shopify apps
embeddings = model.encode(apps_df.description.astype(str).to_list())
print(embeddings.shape)

(4750, 384)


In [23]:
## Creating App2idx 
app2idx = {app_id:idx for idx,app_id in enumerate(apps_df.app_id)}

This model returns embedding of size 384

## Step 2: Define parameters and build Annoy Index

In [17]:
f = embeddings.shape[1] # Length of item vector that will be indexed
t = AnnoyIndex(f, 'angular')  ## Defining index
for idx, vector in enumerate(embeddings):
    t.add_item(idx, vector)

## Caculating trees
t.build(int(np.round(np.sqrt(len(embeddings))))) # Number of trees = sqrt(length of embedding vector)
t.save(str(dataPath/'appEmbedding.ann')) ## saving the annoy index

True

## Step 3:  Search similar apps

In [29]:
def get_similar_apps(app_id, app2idx, app_data, annoy_index, n_items=5):
    query_idx = app2idx[app_id]
    idxs = annoy_index.get_nns_by_item(query_idx, n_items+1)
    return app_data.iloc[idxs,:]

### Some Examples

In [30]:
## Langugage translation apps
get_similar_apps(app_id="9e4748a9-7eda-4814-83b6-0537d44152b1" , app2idx = app2idx, app_data = apps_df, annoy_index=t, n_items=5)

,app_id,url,title,description,tagline,category
0,9e4748a9-7eda-4814-83b6-0537d44152b1,https://apps.shopify.com/translate,Panda Language Translate,Panda Language Translation improve your busine...,Translate your store into multiple languages,Store design
893,be566fe3-2449-4ae5-8b59-06b79a668dff,https://apps.shopify.com/mokool-translate-pro,Translate Pro Using Google,Do you want both more customers and to increas...,Auto Translate to 100 languages in fast. Unlim...,Store design
2409,6de8f41b-f0f6-4e3c-b5b4-731b833741e3,https://apps.shopify.com/easy-language-translate,Easy Language Translate,Translate your store and attract global visito...,Experience a better way to translate store and...,Store design
1396,cccdbfd7-b1fd-4079-8b49-280e19f3a8b9,https://apps.shopify.com/translate-my-store,Translate My Store,Limited Time Offer for the next 500 users: You...,"Translate your store, increase sales in global...",Store design
1892,6281aca5-c6f7-4858-a445-813ec7214abc,https://apps.shopify.com/language-translate-by...,Uplinkly Language Translate,Click ‘Add App’ to start using the translation...,Translate your store & get more sales automati...,Store design
4268,9b6804b6-91d8-4813-a353-f5080155fe56,https://apps.shopify.com/langify,langify,How does langify improve the business of your ...,Translate your shop into multiple languages,Store design


The top row is the query indexed and bottom 5 rows are the similar apps returned by annoy. Looks like translatation app is returning translations

In [36]:
query_id = apps_df.loc[np.random.randint(0, len(apps_df)),'app_id'] ##randomly choosing an app_id
get_similar_apps(app_id=query_id , app2idx = app2idx, app_data = apps_df, annoy_index=t, n_items=5) ## Query for similar apps

,app_id,url,title,description,tagline,category
1896,d957681d-0ceb-4226-bde4-75a5ff4cc10d,https://apps.shopify.com/evmupsell,"Discounted Upsell Bundle, Bogo",==== # Officially the Sellify Apps Alternative...,"Upsellio - In cart upsell, buy x get y, BOGO &...",Sales and conversion optimization
2498,f3b34865-523a-4f88-8e69-95023eed9a37,https://apps.shopify.com/dnt-upsell,Easy Upsell Cross‑sell,Upsell and Cross-sell product app on Shopify b...,"Increase sales with upsell, cross-sell and mak...",Sales and conversion optimization
3847,f718b743-2b12-40d5-8935-db20587ea678,https://apps.shopify.com/in-cart-upsell,In Cart Upsell,"Targeted, Popup-Free Upsells \n In Cart Upsell...",Laser targeted upsell offers in the shopping c...,"Store design,Sales and conversion optimization"
2479,1ee3ad7a-248e-4b32-bb0b-0b9ba76b888d,https://apps.shopify.com/upseller-product-upsell,SellUp • Free Upsell App,Increase your sales and up your upsell game wi...,"Upsell, Bundles, Discounts, Cross-sell & One C...",Sales and conversion optimization
4471,c4359733-0471-47ab-8959-bf6bd569181e,https://apps.shopify.com/product-upsell-manager,Upsell & Cross‑Sell Booster,Features \n \n This app gives you the ability ...,Can show targeted products as an offer to rele...,Sales and conversion optimization
3521,76f39b1f-1c05-4fb0-9616-dd508132349d,https://apps.shopify.com/checkout-booster,Upsell + Related Products,4 in 1 Sales Booster App (with Upsell cross-se...,"Upsell Popup, Recently Viewed Products, BOGO O...",Store design


Upsell Related apps are returning upsell

In [37]:
query_id = apps_df.loc[np.random.randint(0, len(apps_df)),'app_id'] ##randomly choosing an app_id
get_similar_apps(app_id=query_id , app2idx = app2idx, app_data = apps_df, annoy_index=t, n_items=5) ## Query for similar apps

,app_id,url,title,description,tagline,category
1101,5557170e-c5db-44b4-82a5-31db79a7363d,https://apps.shopify.com/predictive-preloader,Instant Page Speeds AI Powered,We use machine learning AI to predict where us...,"Increase Sales, Conv. Rates, Reduce Bounce, 1-...",Store design
3806,5a37f0f7-0090-4750-b4d5-806dd1d74179,https://apps.shopify.com/plug-in-speed,Plug in Speed optimizer,"Click ""Add App"" for a free trial of the code a...","Compress and Optimize Images, CSS & Javascript...",Marketing
939,ccd7adc7-2430-459e-96c2-25ba91cbaf87,https://apps.shopify.com/avada-seo-suite,AVADA SEO Image Optimizer FREE,We guarantee to contribute this app for FREE F...,"FREE FOREVER GUARANTEE! SEO Image Optimizer, A...","Store design,Marketing"
516,0fe33ede-d70c-4369-af6e-d5e6d7d39949,https://apps.shopify.com/jumbo-page-speed-opti...,Jumbo ‑ Page speed optimizer,Jumbo Shopify Page Speed Optimizer. \n Jumbo o...,Page speed optimizer for improved conversion a...,Store design
1374,d305c7f0-7d20-40e3-be85-f97c809cd4c1,https://apps.shopify.com/page-speed-boost,Page Speed+SEO Image Optimizer,High-quality images make your store a nice app...,"Image Compression, Site Speed Optimization, Pa...",Store design
4729,ac6fa8f3-bc49-40c9-9cbb-f974322c799a,https://apps.shopify.com/imageoptimizer,Image Optimizer,"Speed up Your Store, Improve SEO and User Expe...",Image Optimizer: Image Compression and Image S...,Store design


Page seats optimizer apps are returning right results